In [1]:
import torch
from torch import nn
from tqdm import tqdm
import toml
os.chdir("..")

/home/elhanzo/.local/miniconda3/envs/iic/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
from core.dl_framework.utils import read_config
from core.dl_framework.data import data_pipeline
from core.dl_framework.learner import Learner
from pathlib import Path
import numpy as np
from torchvision import transforms
config_file = read_config(toml.load("configs/default_train_config.toml"))
config_file = read_config(toml.load("configs/default_train_config.toml"))
train_data_path = "seg_train/seg_train"
data = data_pipeline(config_file, train_data_path, "train_data", False)
learn = Learner(data, config_file)
# data_norm = next(iter(data.train_dl))


In [11]:
import os
print(os.getenv("HOME"))
if "colab" in os.environ:
    print("hi")
os.environ

/home/elhanzo


environ{'USER': 'elhanzo',
        'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/mmocz/.vscode/extensions/ms-vscode-remote.remote-wsl-0.66.0',
        'SHLVL': '2',
        'WT_PROFILE_ID': '{17bf3de4-5353-5709-bcf9-835bd952a95e}',
        'HOME': '/home/elhanzo',
        'WSL_DISTRO_NAME': 'Ubuntu-22.04',
        'LOGNAME': 'elhanzo',
        'NAME': 'Code',
        'WSL_INTEROP': '/run/WSL/12_interop',
        '_': '/home/elhanzo/.local/miniconda3/envs/iic/bin/python',
        'TERM': 'xterm-color',
        'PATH': '/home/elhanzo/.local/miniconda3/envs/iic/bin:/home/elhanzo/.vscode-server/bin/dfd34e8260c270da74b5c2d86d61aee4b6d56977/bin/remote-cli:/home/elhanzo/.local/miniconda3/envs/iic/bin:/home/elhanzo/.local/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files/Common Files/Oracle/Java/javapath:/mnt/c/Program Files (x86)/Common Files/Intel/Shared Libraries/redist/intel64/compiler:/mnt/c/WINDOW

In [33]:
learn.fit(3)

100%|██████████| 44/44 [00:01<00:00, 25.67it/s]


epoch: 1/3	[valid_acc: 0.5744	valid_acc_per_class: [0.40952826 0.75678116 0.5915685  0.6441189  0.3845187  0.6359284 ]	valid_loss: 1.4675	train_loss: 1.7330	train_time: 00:00:13]
best checkpoint: valid_loss: 1.4674860482866114


100%|██████████| 44/44 [00:01<00:00, 23.55it/s]


epoch: 2/3	[valid_acc: 0.6192	valid_acc_per_class: [0.46368554 0.80627817 0.6319679  0.63639075 0.46347705 0.6951172 ]	valid_loss: 1.3225	train_loss: 1.3796	train_time: 00:00:27]
best checkpoint: valid_loss: 1.3225062408230521


100%|██████████| 44/44 [00:01<00:00, 24.79it/s]


epoch: 3/3	[valid_acc: 0.6495	valid_acc_per_class: [0.50266355 0.8466093  0.65599275 0.6398561  0.526092   0.7123743 ]	valid_loss: 1.2545	train_loss: 1.2796	train_time: 00:00:40]
best checkpoint: valid_loss: 1.2544895898212085


In [6]:
xb, yb = next(iter(learn.learn.data.train_dl))
xb, yb = xb.to(learn.device), yb.to(learn.device)
out = learn.learn.model(xb)

In [71]:
# def get_missmatch_sample(out):
    # _, pred = torch.max(out.data, 1)
    
    # return pred
# get_missmatch_sample(out)
learn.learn.model.eval()
num_classes = learn.learn.data.c

with torch.no_grad():
    pbar_valid_dl = tqdm(learn.learn.data.valid_dl, total=len(learn.learn.data.valid_dl))
    missmatch_dict = dict(zip(range(learn.learn.data.c),[[] for c in range(learn.learn.data.c)]))
    for xb, yb, sampler_idx in pbar_valid_dl:
        xb, yb = xb.to(learn.device), yb.to(learn.device)
        out = learn.learn.model(xb)
        _, pred = torch.max(out.data, 1)

        missmatch_mask = ~(pred == yb)
        print(learn.learn.data.train_dl.dataset.y[sampler_idx[missmatch_mask]], pred[missmatch_mask].cpu().numpy())
        break
# print(missmatch_dict)

  0%|          | 0/44 [00:00<?, ?it/s]

[4 0 0 5 4 2 4 3 0 0 4 0 4 1 1 5 4 0 5 0 3 4 0 0] [3 4 3 0 0 3 2 2 5 5 2 3 2 0 4 4 5 1 3 3 4 3 3 3]


In [82]:
missmatch_dict[(0,1)]

KeyError: (0, 1)

In [59]:
~pred == yb

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False], device='cuda:0')

In [19]:
dict(zip(range(6),[[] for i in range(6)]))

{0: [], 1: [], 2: [], 3: [], 4: [], 5: []}

In [25]:
from core.dl_framework.data import CustomDataset
from core.dl_framework.data import get_transforms
data_path = (Path(config_file["p_local_data_path"]) / "processed_files" / "train_data" ).with_suffix(".h5")
transform = get_transforms(config_file)
dataset = CustomDataset(data_path, transform)

AttributeError: 'tuple' object has no attribute 'y'

In [169]:
from core.dl_framework.utils import get_specific_history
from core.dl_framework.utils import plot_history_all
best_history = get_specific_history(Path(config_file["c_c_ckp_path"]), monitor="valid_acc", browse_all_files=True)
best_model_dict = torch.load(best_history[-1])
learn.learn.model.load_state_dict(best_model_dict["state_dict"])
learn.learn.opt.load_state_dict(best_model_dict["optimizer"])

In [208]:
from torch.utils.data import DataLoader
def get_mean_and_std(dataloader):
    pbar_dl = tqdm(dataloader, total=len(dataloader))
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for i, (xb, yb) in enumerate(pbar_dl):
        channels_sum += torch.mean(xb, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(xb**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean**2)**0.5
    return mean, std

# DataLoader(len(data.train_dl.dataset), batch_size=64)
# channels_sum, channels_squared_sum, num_batches = 0, 0, 0
# pbar_ds = tqdm(data.train_dl.dataset, total=len(data.train_dl.dataset))

# for xs, ys in pbar_ds:
#     channels_sum += torch.mean(xb, dim=[0, 2, 3])
#     channels_squared_sum += torch.mean(xb**2, dim=[0, 2, 3])
#     num_batches += 1

# mean = channels_sum / num_batches
# std = (channels_squared_sum / num_batches - mean**2)**0.5

100%|██████████| 14034/14034 [00:42<00:00, 329.55it/s]


In [173]:
def test_arch(learn, model):
    valid_dl = learn.learn.data.valid_dl
    device = learn.device
    model = model.to(device)
    criterion = learn.learn.loss_func
    model.eval()
    with torch.no_grad():
        correct = 0
        pbar_valid_dl = tqdm(valid_dl, total=len(valid_dl))
        for xb, yb in pbar_valid_dl:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, pred = torch.max(out.data, 1)
            correct += (pred == yb).sum().item() / len(yb)
            loss = criterion.calc(out, yb)
        print(correct/len(valid_dl))
        

test_arch(learn, learn.learn.model)

100%|██████████| 44/44 [00:01<00:00, 24.57it/s]

0.8532283057851239


In [175]:
best_model_dict

{'best_values': {'valid_acc': [<ufunc 'greater'>,
   0.8528731921487602,
   0.8532283057851239],
  'valid_loss': [<ufunc 'less'>, 0.7601445479826494, 0.7601443691687151],
  'train_loss': [<ufunc 'less'>, 0.38256853699684146, 0.38102036765643527]},
 'history_raw': {'epochs': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   10

In [5]:
plot_history_all(best_history)
# best_history[0][0]

In [43]:
import torchvision
pretrained = True
batch = next(iter(data.train_dl))
img = batch[0].permute(0, 3, 2, 1)
model = torchvision.models.resnet18(pretrained)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 6)

model(img/255)
# import matplotlib.pyplot as plt
# plt.imshow(img.permute(0, 1, 2))


tensor([[ 2.9437e-02,  5.6660e-01,  4.3710e-01, -2.4919e-01,  5.2946e-01,
          1.5116e+00],
        [-2.1430e-01, -6.8146e-01, -3.8934e-01, -4.1285e-01,  4.3214e-02,
          9.3181e-01],
        [ 2.8854e-01, -2.7958e-01, -2.1406e-01, -1.4422e+00,  1.7665e-02,
          1.0402e+00],
        [ 7.6974e-01, -1.1695e-02,  4.0755e-01, -1.0151e+00, -2.5027e-01,
          7.6901e-01],
        [-9.4725e-02,  1.0334e+00,  1.7287e+00, -1.1007e+00,  1.6617e-01,
          8.1947e-01],
        [ 1.2263e-01, -2.3190e-01,  2.1097e-01, -4.7100e-01,  6.9263e-01,
          7.1590e-01],
        [ 1.3604e-01,  1.0653e-01,  7.0132e-01,  1.3013e+00, -2.6025e-01,
          4.0852e-01],
        [-5.1715e-01,  2.5876e-01,  3.2860e-01, -4.8839e-01, -5.5886e-01,
          6.0666e-01],
        [-1.3317e-01, -1.9250e-01,  6.1959e-01, -1.0243e+00,  1.5129e-01,
          6.5095e-01],
        [-9.0879e-02, -3.3638e-02, -4.4355e-01, -1.9576e-01, -4.7457e-01,
          6.9916e-01],
        [-3.6090e-01, -1.4958e

In [8]:
import h5py
from pathlib import Path
data_path = Path(config_file["p_local_data_path"]) / "processed_files" / "train_data.h5"
dset = h5py.File(data_path, "r") 

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [133]:

    
img = next(iter(data_test.train_dl))[0]
# net = CustomModel(img.size(1), 6, 16, 124, 4)
# img.size(2)
data_test.train_dl.dataset[0][0].shape
learn.learn.model(img)


tensor([[-2.9168e-01],
        [-1.6196e-01],
        [ 2.9472e-01],
        [ 6.2239e-02],
        [ 1.9767e-01],
        [-4.6405e-01],
        [-8.1470e-01],
        [-2.6797e-01],
        [-3.7364e-01],
        [ 1.8129e-01],
        [ 8.5726e-02],
        [ 5.2427e-01],
        [ 5.0081e-02],
        [-6.7159e-01],
        [ 2.2814e-01],
        [-8.1703e-02],
        [-1.9749e-01],
        [ 3.3920e-02],
        [ 2.7294e-01],
        [ 6.6710e-04],
        [ 9.9744e-02],
        [-5.0846e-01],
        [-1.0967e-01],
        [-1.8858e-02],
        [ 8.7219e-01],
        [ 2.1678e-01],
        [ 3.8744e-01],
        [ 1.8138e-01],
        [-1.9797e-02],
        [ 7.9672e-01],
        [ 2.2826e-01],
        [ 1.5312e-01],
        [ 1.1312e-01],
        [-1.7619e-01],
        [ 2.5308e-02],
        [-3.1693e-01],
        [ 3.3003e-01],
        [-5.1250e-01],
        [-1.9844e-02],
        [-6.8867e-01],
        [-8.9160e-01],
        [-3.8979e-01],
        [ 3.8396e-01],
        [ 2

In [114]:
img = next(iter(data_test.train_dl))[0]
img.shape

torch.Size([64, 3, 124, 124])

In [126]:
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)
conv_output_size(124,3,1,1)
conv_output_size(124, 2, 0, 2)
62*62*16
# conv_output_size(1984, 1, 2)

61504